In [1]:
import sys
import random
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import sumolib
import traci
from sumolib import checkBinary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout
import matplotlib.pyplot as plt
import pandas as pd
import os
import math


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

sumoBinary = checkBinary('sumo-gui')
# sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

/home/andyfu/miniconda3/envs/sumo/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/andyfu/miniconda3/envs/sumo/lib/python3.10/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /opt/conda/conda-bld/pytorch_1712608839953/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


SUMO_HOME found
Using device: cpu


In [2]:
def intervehicleConnectivity(threshold = None):
    xs = []
    ys = []
    for vehicle in traci.vehicle.getIDList():
        x, y = traci.vehicle.getPosition(vehicle)
        xs.append(x)
        ys.append(y)
    xs = torch.tensor(xs, dtype=torch.float32).to(device).view(-1,1)
    ys = torch.tensor(ys, dtype=torch.float32).to(device).view(-1,1)
    intervehicle_distances = torch.sqrt((xs - xs.t())**2 + (ys - ys.t())**2)
    if threshold is not None:
        # # make only the distances less than the threshold non-zero, the rest
        # intervehicle_distances = torch.where(intervehicle_distances < threshold, intervehicle_distances, torch.zeros_like(intervehicle_distances))
        # make the distances 1 if less than the threshold, 0 otherwise
        intervehicle_distances = torch.where(intervehicle_distances < threshold, torch.ones_like(intervehicle_distances), torch.zeros_like(intervehicle_distances))
    return intervehicle_distances, xs, ys

In [3]:
def randomTrips(dur=1000, density=12):
    os.system("python $SUMO_HOME/tools/randomTrips.py -n config/osm.net.xml.gz -r config/osm.passenger.trips.xml -e " + str(dur) + " -l --insertion-density=" + str(density))

def shouldContinueSim():
    numVehicles = traci.simulation.getMinExpectedNumber()
    return True if numVehicles > 0 else False

def restart(dur, density):
    with io.StringIO() as buf, redirect_stdout(buf):
        try:
            traci.close()
        except:
            pass
        randomTrips(dur, density)
        traci.start(sumoCmd)

def close():
    traci.close()

In [4]:
import copy

class Knowledges:
    def __init__(self):
        self.knowledges = {}
        self.delays = {}
    
    def add_observations(self, vehicles, observed_vehicles):
        for vehicle, visibility in zip(vehicles, observed_vehicles):
            if vehicle not in self.knowledges:
                self.knowledges[vehicle] = []
                self.delays[vehicle] = 0
            self.knowledges[vehicle].append(int(visibility))
            if visibility == 0:
                self.delays[vehicle] += 1
            else:
                self.delays[vehicle] = 0
    
    def merge_knowledges(self, new_knowledges, new_delays):
        for vehicle, visibility in new_knowledges.items():
            if vehicle not in self.knowledges:
                self.knowledges[vehicle] = copy.deepcopy(visibility)
                self.delays[vehicle] = new_delays[vehicle]
            else:
                for i in range(1, len(self.knowledges[vehicle])+1):
                    if i > len(visibility):
                        break
                    self.knowledges[vehicle][-i] = visibility[-i] | self.knowledges[vehicle][-i]
                self.delays[vehicle] = min(self.delays[vehicle], new_delays[vehicle])
        return copy.deepcopy(self.knowledges), copy.deepcopy(self.delays)

    def get_knowledges(self):
        return copy.deepcopy(self.knowledges)
    
    def get_delays(self):
        return copy.deepcopy(self.delays)
    
    def evaluate_knowledge(self, large_delay_threshold = 10):
        observed = 0
        total = 0
        delay = 0
        large_delay = 0
        num_vehicles = len(self.knowledges)
        for vehicle, visibility in self.knowledges.items():
            observed += sum(visibility)
            total += len(visibility)
            delay += self.delays[vehicle]
            if self.delays[vehicle] >= large_delay_threshold:
                large_delay += 1
        return 1-(observed / total), delay / num_vehicles, large_delay / num_vehicles

class Beacon:
    def __init__(self, trace_hidden):
        self.trace_hidden = copy.deepcopy(trace_hidden)
    
    def update(self, trace_hidden):
        self.trace_hidden = copy.deepcopy(trace_hidden)


In [11]:
# find the maximum action space
restart(1000, 6)
max_action_space = 0
vehicle_knowledges = {}
total_states = 0

total_missing = 0
total_delay = 0
total_large_delay = 0

while shouldContinueSim():
    traci.simulationStep()
    ids = traci.vehicle.getIDList()
    connectivity, xs, ys = intervehicleConnectivity(800)
    # minus the diagonal
    action_spaces = connectivity - torch.eye(connectivity.size(0)).to(device)
    
    for i, vehicle in enumerate(ids):
        if vehicle not in vehicle_knowledges:
            vehicle_knowledges[vehicle] = Knowledges()
        vehicle_knowledges[vehicle].add_observations(ids, connectivity[i])
    # for i, vehicle in enumerate(ids):
    #     # get non-zero indices except the diagonal
    #     non_zero_indices = np.where(action_spaces[i] == 1)[0]
    #     for index in non_zero_indices:
    #         receiver = ids[index]
    #         assert vehicle in vehicle_knowledges and receiver in vehicle_knowledges
    #         vehicle_knowledges[receiver].merge_knowledges(vehicle_knowledges[vehicle].get_knowledges(), vehicle_knowledges[vehicle].get_delays())
        # if len(non_zero_indices) > 0:
        #     selected_index = random.choice(non_zero_indices)
        #     receiver = ids[selected_index]
        #     assert vehicle in vehicle_knowledges and receiver in vehicle_knowledges
        #     vehicle_knowledges[receiver].merge_knowledges(vehicle_knowledges[vehicle].get_knowledges(), vehicle_knowledges[vehicle].get_delays())
    for i, vehicle in enumerate(ids):
        total_states += 1
        missing, delay, large_delay = vehicle_knowledges[vehicle].evaluate_knowledge()
        total_missing += missing
        total_delay += delay
        total_large_delay += large_delay
print("Average missing: ", total_missing / total_states)
print("Average delay: ", total_delay / total_states)
print("Average large delay: ", total_large_delay / total_states)

Success.
Average missing:  0.027813125753645893
Average delay:  0.38216493308263394
Average large delay:  0.008108149334431105


In [9]:
num_vehicles = len(vehicle_knowledges)
total_missing = 0
total_delay = 0
step = 0
max_delay = 0 
for vehicle, knowledge in vehicle_knowledges.items():
    m, d, l = knowledge.evaluate_knowledge() 
    total_missing += m
    total_delay += d
    max_delay = max(max_delay, l)
print("Average missing: ", total_missing/num_vehicles)
print("Average delay: ", total_delay/num_vehicles)
print("Max delay: ", max_delay)

Average missing:  0.03477086122808777
Average delay:  1.5325179224318364
Max delay:  0.6181818181818182
